In [11]:
# ==========================================================
# 1. IMPORTS
# ==========================================================
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# ==========================================================
# 2. LOAD DATA
# ==========================================================
interactions = pd.read_csv("/learning_interactions_dataset_updated.csv")
items = pd.read_csv("/python learning path.csv")

# Ensure original user IDs like U0142 are strings
interactions["user_id"] = interactions["user_id"].astype(str)

interactions["timestamp"] = pd.to_datetime(interactions["timestamp"])

print("Interactions:", interactions.shape)
print("Items:", items.shape)

# ==========================================================
# 3. BASIC EDA
# ==========================================================
print("\nUsers:", interactions["user_id"].nunique())
print("Items:", interactions["item_id"].nunique())
print("Interactions:", len(interactions))

print("\nAction counts:")
print(interactions["action"].value_counts())

print("\nTime range:")
print(interactions["timestamp"].min(), "→", interactions["timestamp"].max())

# ==========================================================
# 4. FEATURE ENGINEERING
# ==========================================================
# implicit label
interactions["label"] = (
    interactions["action"].isin(["completed","attempted"]) |
    (interactions["rating_score"] > 0)
).astype(int)

# time features
interactions["hour"] = interactions["timestamp"].dt.hour
interactions["dow"] = interactions["timestamp"].dt.dayofweek

# merge metadata
interactions = interactions.merge(
    items,
    left_on="course_topic",
    right_on="Topic",
    how="left"
)

# sort
interactions = interactions.sort_values(["user_id","timestamp"])

# encode ids, keep original strings
users = interactions["user_id"].unique().tolist()
items_list = interactions["item_id"].unique().tolist()

user2idx = {u:i for i,u in enumerate(users)}
idx2user = {i:u for u,i in user2idx.items()}

item2idx = {i:j for j,i in enumerate(items_list)}
idx2item = {v:k for k,v in item2idx.items()}

interactions["u"] = interactions["user_id"].map(user2idx)
interactions["i"] = interactions["item_id"].map(item2idx)

n_users = len(user2idx)
n_items = len(item2idx)

# mapping item index → topic
itemid_to_topic = interactions.set_index("item_id")["course_topic"].to_dict()
idx2topic = {idx: itemid_to_topic[itemid] for idx, itemid in idx2item.items()}

print("\nEncoded Users:", n_users, "Encoded Items:", n_items)

# ==========================================================
# 5. DATASET CLASS — SEQUENCE BUILDER
# ==========================================================
class SeqDataset(Dataset):
    def __init__(self, df, max_len=20):
        self.samples = []
        df = df.sort_values(["u","timestamp"])
        for u, grp in df.groupby("u"):
            items = grp["i"].tolist()
            for t in range(1, len(items)):
                seq = items[max(0, t-max_len):t]
                target = items[t]
                self.samples.append((seq, target))
        self.max_len = max_len

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        seq, target = self.samples[idx]
        seq = np.array(seq, dtype=np.int64)
        pad_len = self.max_len - len(seq)
        if pad_len > 0:
            seq = np.pad(seq, (pad_len, 0))
        return torch.tensor(seq, dtype=torch.long), torch.tensor(target, dtype=torch.long)

train_ds = SeqDataset(interactions)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)

print("Training samples:", len(train_ds))

# ==========================================================
# 6. GRU MODEL
# ==========================================================
class GRURec(nn.Module):
    def __init__(self, n_items, embed_dim=64, hidden_dim=64):
        super().__init__()
        self.emb = nn.Embedding(n_items, embed_dim)
        self.gru = nn.GRU(embed_dim, hidden_dim, batch_first=True)
        self.out = nn.Linear(hidden_dim, n_items)

    def forward(self, seq):
        e = self.emb(seq)
        _, h = self.gru(e)
        return self.out(h[-1])

model = GRURec(n_items).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

# ==========================================================
# 7. TRAINING LOOP
# ==========================================================
EPOCHS = 5

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for seq, tgt in tqdm(train_loader, desc=f"Epoch {epoch}"):
        seq, tgt = seq.to(device), tgt.to(device)

        logits = model(seq)
        loss = loss_fn(logits, tgt)

        opt.zero_grad()
        loss.backward()
        opt.step()

        total_loss += loss.item()

    print(f"Epoch {epoch} Loss: {total_loss/len(train_loader):.4f}")

torch.save(model.state_dict(), "gru_model.pt")
print("\nModel saved as: gru_model.pt")

# ==========================================================
# 8. RECOMMENDATION FUNCTION (TOPICS)
# ==========================================================
def recommend_next_topics(user_id, k=5):

    user_id = str(user_id)   # Ensure string like U0142

    df = interactions[interactions["user_id"] == user_id].sort_values("timestamp")

    # Need at least 2 interactions
    if len(df) < 2:
        return None

    # current topic
    current_item_id = df.iloc[-1]["item_id"]
    current_topic = itemid_to_topic.get(current_item_id, "Unknown")

    # build sequence
    seq = df["i"].tolist()[-20:]
    seq = np.pad(seq, (20-len(seq), 0))
    seq = torch.tensor(seq, dtype=torch.long).unsqueeze(0).to(device)

    # predict
    logits = model(seq)[0].detach().cpu().numpy()
    topk = np.argsort(-logits)[:k]

    next_topics = [idx2topic[i] for i in topk]
    return current_topic, next_topics

# ==========================================================
# 9. NICE PRINT FUNCTION
# ==========================================================
def get_user_current_and_next(user_id):
    result = recommend_next_topics(user_id, k=5)

    if result is None:
        print(f"\n❌ User {user_id} does NOT have enough interactions (need ≥2).")
        return

    current_topic, next_topics = result

    print(f"\n🧑 User ID: {user_id}")
    print(f"📌 Current Topic: {current_topic}")

    print("\n🔮 Next Recommended Topics:")
    for i, t in enumerate(next_topics, 1):
        print(f"{i}. {t}")

# ==========================================================
# 10. ASK USER FOR REAL ID LIKE U0142
# ==========================================================
user_input = input("\nEnter User ID (example: U0142): ")
get_user_current_and_next(user_input)


Using device: cpu
Interactions: (1000, 13)
Items: (38, 3)

Users: 198
Items: 150
Interactions: 1000

Action counts:
action
viewed       211
completed    207
started      198
skipped      196
attempted    188
Name: count, dtype: int64

Time range:
2025-01-01 00:44:59 → 2025-12-13 18:20:13

Encoded Users: 198 Encoded Items: 150
Training samples: 802


Epoch 0: 100%|██████████| 13/13 [00:00<00:00, 47.89it/s]


Epoch 0 Loss: 5.0334


Epoch 1: 100%|██████████| 13/13 [00:00<00:00, 45.71it/s]


Epoch 1 Loss: 4.9547


...

Epoch 2 Loss: 4.8933


Epoch 3: 100%|██████████| 13/13 [00:00<00:00, 45.15it/s]


Epoch 3 Loss: 4.8317


Epoch 4: 100%|██████████| 13/13 [00:00<00:00, 51.45it/s]


Epoch 4 Loss: 4.7705

Model saved as: gru_model.pt

Enter User ID (example: U0142): U0142

🧑 User ID: U0142
📌 Current Topic: Probability Distributions

🔮 Next Recommended Topics:
1. Descriptive Statistics
2. Control Flow and Loops
3. Hypothesis Testing
4. Introduction to Robotics
5. Probability Distributions
